In [4]:
from keras.layers import Input, Dense
from keras.models import Model

In [5]:
encoding_dim = 32

In [6]:
input_img = Input(shape=(784,))
encoded = Dense(encoding_dim, activation='relu')(input_img)
decoded = Dense(784, activation='sigmoid')(encoded)

In [8]:
# end-to-end reconstruction
autoencoder = Model(input_img, decoded) 

# map input to encoded representation
encoder = Model(input_img, encoded) 

# decode a representation
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [9]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [10]:
from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

In [12]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

(60000, 784)
(10000, 784)


In [13]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 4s 62us/step - loss: 0.3576 - val_loss: 0.2717
Epoch 2/50
60000/60000 [==============================] - 4s 59us/step - loss: 0.2653 - val_loss: 0.2547
Epoch 3/50
60000/60000 [==============================] - 4s 59us/step - loss: 0.2436 - val_loss: 0.2303
Epoch 4/50
60000/60000 [==============================] - 4s 59us/step - loss: 0.2224 - val_loss: 0.2123
Epoch 5/50
60000/60000 [==============================] - 4s 59us/step - loss: 0.2071 - val_loss: 0.1992
Epoch 6/50
60000/60000 [==============================] - 4s 59us/step - loss: 0.1958 - val_loss: 0.1895
Epoch 7/50
60000/60000 [==============================] - 4s 59us/step - loss: 0.1872 - val_loss: 0.1819
Epoch 8/50
60000/60000 [==============================] - 4s 59us/step - loss: 0.1802 - val_loss: 0.1753
Epoch 9/50
60000/60000 [==============================] - 4s 59us/step - loss: 0.1740 - val_loss: 0.1695
Epoch